In [ ]:
import numpy as np
import scipy
import random
import matplotlib.pyplot as plt

```% Created by Eugene M. Izhikevich, February 25, 2003

% Excitatory neurons Inhibitory neurons
Ne=800; Ni=200;
re=rand(Ne,1); ri=rand(Ni,1);

a=[0.02*ones(Ne,1); 0.02+0.08*ri];
b=[0.2*ones(Ne,1); 0.25-0.05*ri];
c=[-65+15*re.^2; -65*ones(Ni,1)];
d=[8-6*re.^2; 2*ones(Ni,1)];

S=[0.5*rand(Ne+Ni,Ne), -rand(Ne+Ni,Ni)];

v=-65*ones(Ne+Ni,1); % Initial values of v
u=b.*v; % Initial values of u

firings=[]; % spike timings

for t=1:1000 % simulation of 1000 ms

    I=[5*randn(Ne,1);2*randn(Ni,1)]; % thalamic input
    
    fired=find(v>=30); % indices of spikes
    firings=[firings; t+0*fired,fired];

    v(fired)=c(fired);
    u(fired)=u(fired)+d(fired);

    I=I+sum(S(:,fired),2);

    v=v+0.5*(0.04*v.^2+5*v+140-u+I); % step 0.5 ms
    v=v+0.5*(0.04*v.^2+5*v+140-u+I); % for numerical

    u=u+a.*(b.*v-u); % stability
end;


In [ ]:
# Parámetros y condiciones iniciales

np.random.seed(42)
print("Primeros ruidos:", np.random.randn(5))

# Definimos Neuronas (1000 - 80% e, 20% i)
Ne = int(800)  # Excitatory neurons
Ni = int(200)  # Inhibitory neurons

# Inicializamos aleatoriamente
re = np.random.rand(Ne)
ri = np.random.rand(Ni)

# Definimos las matrices de parametros para (Excit - Inhib)
a = np.concatenate([0.02*np.ones(Ne), 0.02+0.08*ri])
b = np.concatenate([0.2*np.ones(Ne), 0.25-0.05*ri])
c = np.concatenate([-65+15*re**2, -65*np.ones(Ni)])
d = np.concatenate([8-6*re**2, 2*np.ones(Ni)])

#k = 5e5/(Ne+Ni)**2

k = 0.5

# Definimos la matriz de acoplamiento (random) (0.5 para excitatorias y -1 para inhib)
S = np.concatenate([k * np.random.rand(Ne + Ni, Ne), 
                    -2*k*np.random.rand(Ne + Ni, Ni)], axis=1)

# Condiciones iniciales
v = -65*np.ones(Ne+Ni) #+ 5*np.random.randn(Ne+Ni) #-65*np.ones(Ne+Ni) # Initial values of v
u = b*v # Initial values of u

# Lista de picos
firings = [] # spike timings

In [ ]:
plt.imshow(S) 

In [ ]:
plt.hist(re**2), plt.hist(ri)

In [ ]:
plt.plot(range(0,1000), np.concatenate([5 * np.random.randn(Ne), 2 * np.random.randn(Ni)]))

In [ ]:
# Python original
print("Pesos E→E originales:", S[:5, :5])  # Primeras 5x5
print("Pesos I→E originales:", S[800:805, :5])

In [ ]:
# Simulation of 1000 ms

potentials = [v]
I_thalamic_history = []
I_synaptic_history = []
I_total_history = []

for t in range(1000):
    
    # Input talámico (ya lo tienes)
    I_thalamic = np.concatenate([5 * np.random.randn(Ne), 2 * np.random.randn(Ni)])
    # I = np.concatenate([1.8 * np.random.poisson(1,Ne), 1 * np.random.poisson(1,Ni)])
    
    # Inicializar corriente sináptica
    I_synaptic = np.zeros(Ne + Ni)

    fired = np.where(v >= 30)[0]  # indices of spikes
    
    if len(fired) > 0:
        firings.extend([[t, neuron] for neuron in fired])
        v[fired] = c[fired]
        u[fired] = u[fired] + d[fired]
        I_synaptic = np.sum(S[:, fired], axis=1)
    
    # Corriente total
    I_total = I_thalamic + I_synaptic
    
    # Two half-steps for numerical stability
    v = v + 0.5 * (0.04 * v**2 + 5 * v + 140 - u + I_total)
    v = v + 0.5 * (0.04 * v**2 + 5 * v + 140 - u + I_total)
    u = u + a * (b * v - u)


    potentials.append(v)
    I_thalamic_history.append(I_thalamic.copy())
    I_synaptic_history.append(I_synaptic.copy())
    I_total_history.append(I_total.copy())

# Convert firings to array for plotting
if firings:
    firings = np.array(firings)
    plt.figure(figsize=(10, 6))
    plt.plot(firings[:, 0], firings[:, 1], '.', markersize=0.8, color = 'black')
    plt.xlabel('Time (ms)')
    plt.ylabel('Neuron index')
    plt.title('Izhikevich Network Spike Raster')
    plt.show()
else:
    print("No spikes recorded")
    

In [ ]:
# Convertir listas a arrays
I_thalamic_matlab = np.array(I_thalamic_history)  # (1000, 1000)
I_synaptic_matlab = np.array(I_synaptic_history)  # (1000, 1000)
I_total_matlab = np.array(I_total_history)        # (1000, 1000)

# Separar excitatorias e inhibitorias
I_thal_exc_matlab = I_thalamic_matlab[:, :800]    # (1000, 800)
I_thal_inh_matlab = I_thalamic_matlab[:, 800:]    # (1000, 200)
I_syn_exc_matlab = I_synaptic_matlab[:, :800]     # (1000, 800)
I_syn_inh_matlab = I_synaptic_matlab[:, 800:]     # (1000, 200)
I_tot_exc_matlab = I_total_matlab[:, :800]        # (1000, 800)
I_tot_inh_matlab = I_total_matlab[:, 800:]        # (1000, 200)

# Estadísticas MATLAB
stats_matlab = {
    'thalamic_exc': {
        'mean_per_neuron': np.mean(I_thal_exc_matlab, axis=0),  # Media temporal por neurona
        'std_per_neuron': np.std(I_thal_exc_matlab, axis=0),    # Std temporal por neurona
        'mean_per_timestep': np.mean(I_thal_exc_matlab, axis=1), # Media poblacional por timestep
        'min_per_timestep': np.min(I_thal_exc_matlab, axis=1),
        'max_per_timestep': np.max(I_thal_exc_matlab, axis=1)
    },
    'synaptic_exc': {
        'mean_per_neuron': np.mean(I_syn_exc_matlab, axis=0),
        'std_per_neuron': np.std(I_syn_exc_matlab, axis=0),
        'mean_per_timestep': np.mean(I_syn_exc_matlab, axis=1),
        'min_per_timestep': np.min(I_syn_exc_matlab, axis=1),
        'max_per_timestep': np.max(I_syn_exc_matlab, axis=1)
    },
    'total_exc': {
        'mean_per_neuron': np.mean(I_tot_exc_matlab, axis=0),
        'std_per_neuron': np.std(I_tot_exc_matlab, axis=0),
        'mean_per_timestep': np.mean(I_tot_exc_matlab, axis=1),
        'min_per_timestep': np.min(I_tot_exc_matlab, axis=1),
        'max_per_timestep': np.max(I_tot_exc_matlab, axis=1)
    },
    'thalamic_inh': {
        'mean_per_neuron': np.mean(I_thal_inh_matlab, axis=0),  # Media temporal por neurona
        'std_per_neuron': np.std(I_thal_inh_matlab, axis=0),    # Std temporal por neurona
        'mean_per_timestep': np.mean(I_thal_inh_matlab, axis=1), # Media poblacional por timestep
        'min_per_timestep': np.min(I_thal_inh_matlab, axis=1),
        'max_per_timestep': np.max(I_thal_inh_matlab, axis=1)
    },
    'synaptic_inh': {
        'mean_per_neuron': np.mean(I_syn_inh_matlab, axis=0),
        'std_per_neuron': np.std(I_syn_inh_matlab, axis=0),
        'mean_per_timestep': np.mean(I_syn_inh_matlab, axis=1),
        'min_per_timestep': np.min(I_syn_inh_matlab, axis=1),
        'max_per_timestep': np.max(I_syn_inh_matlab, axis=1)
    },
    'total_inh': {
        'mean_per_neuron': np.mean(I_tot_inh_matlab, axis=0),
        'std_per_neuron': np.std(I_tot_inh_matlab, axis=0),
        'mean_per_timestep': np.mean(I_tot_inh_matlab, axis=1),
        'min_per_timestep': np.min(I_tot_inh_matlab, axis=1),
        'max_per_timestep': np.max(I_tot_inh_matlab, axis=1)
    }
}

In [ ]:
# Comparar medias
print("=== INPUT TALÁMICO ===")
print(f"MATLAB exc - Media por neurona: {np.mean(stats_matlab['thalamic_exc']['mean_per_neuron']):.4f}")

print(f"MATLAB exc - Std por neurona: {np.mean(stats_matlab['thalamic_exc']['std_per_neuron']):.4f}")


print("\n=== INPUT SINÁPTICO ===")
print(f"MATLAB exc - Media: {np.mean(stats_matlab['synaptic_exc']['mean_per_neuron']):.4f}")

print(f"MATLAB exc - Std: {np.mean(stats_matlab['synaptic_exc']['std_per_neuron']):.4f}")


# Rangos de variación
print(f"MATLAB exc - Rango típico: [{np.mean(stats_matlab['synaptic_exc']['min_per_timestep']):.2f}, {np.mean(stats_matlab['synaptic_exc']['max_per_timestep']):.2f}]")


In [ ]:
plt.plot(range(0,1000), np.array(currents)[:,:800].mean(axis=1) , '--')

In [ ]:
# Python: después del cálculo sináptico
print("Input total neurona 0:", np.array(currents)[0,0])
print("Rango inputs en t_final:", np.min(np.array(currents)[0,:800]), np.max(np.array(currents)[0,:]))

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1000), np.array(potentials)[1:,10], '--', linewidth=0.3, color = 'black')
plt.xlabel('Time (ms)')
plt.ylabel('V (mV)')
plt.title('Izhikevich Excitatory Neuron')
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(range(1000), np.array(potentials)[1:,-1], '--', linewidth=0.3, color = 'black')
plt.xlabel('Time (ms)')
plt.ylabel('V (mV)')
plt.title('Izhikevich Inhibitory Neuron')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

plt.title('Distribución de Parámetros (Fig. 2)')
# 3. Distribución de parámetros c,d (como Fig. 2 del paper)
plt.subplot(2, 2, 1)
plt.hist(a[:800], alpha=0.4, color='red', bins= 10, label='Excitatorias a')
plt.hist(b[:800], alpha=0.4, color='blue', bins= 10,label='Excitatorias b')
plt.legend()
plt.subplot(2, 2, 2)
plt.hist(c[:800], alpha=0.4, color='green', bins= 5,label='Excitatorias c')
plt.hist(d[:800], alpha=0.4, color='orange', bins= 5,label='Excitatorias d')

plt.subplot(2, 2, 3)
plt.hist(a[800:], alpha=0.4, color='red', bins= 10,label='Inhibitorias a')
plt.hist(b[800:], alpha=0.4, color='blue', bins= 10,label='Inhibitorias b')
plt.subplot(2, 2, 4)
plt.hist(c[800:], alpha=0.4, color='green', bins= 2,label='Inhibitorias c')
plt.hist(d[800:], alpha=0.4, color='orange', bins= 2,label='Inhibitorias d')


plt.legend()

In [ ]:
plt.figure(figsize=(8, 6))
# 4. Histograma de frecuencias
plt.subplot(1,1,1)
freq_exc = []
freq_inh = []

for i in range(0,Ne):
    spike_count = sum(firings[:,1]==i)
    freq_exc.append(spike_count)

for i in range(Ne,Ne+Ni):
    spike_count = sum(firings[:,1]==i) 
    freq_inh.append(spike_count)

plt.hist(freq_exc, bins=30, alpha=0.7, density=True, label='Excitatorias')
plt.hist(freq_inh, bins=30, alpha=0.7, density=True, label='Inhibitorias')
plt.xlabel('Spikes en 1000ms')
plt.ylabel('Densidad')
plt.title('Distribución de Actividad')
plt.legend()

In [ ]:
# Análisis adicional para comparar con Brian2
# Actividad poblacional
bin_size = 5  # ms
time_bins = np.arange(0, 1000, bin_size)
activity = []
for t in time_bins:
    spikes_in_bin = np.sum((firings[:, 0] >= t) & (firings[:, 0] < t + bin_size))
    activity.append(spikes_in_bin)

plt.figure(figsize=(8, 6))

# Actividad de red
plt.subplot(1, 1, 1)
plt.plot(time_bins, activity, 'k-', linewidth=1)
plt.xlabel('Time (ms)')
plt.ylabel(f'Spikes/{bin_size}ms')
plt.title('Network Activity')

In [ ]:
# Análisis espectral
plt.subplot(1, 1, 1)
from scipy import signal
freqs, psd = signal.periodogram(activity, fs=1000/bin_size)
alpha_band = (freqs >= 8) & (freqs <= 12)
gamma_band = (freqs >= 30) & (freqs <= 50)

plt.plot(freqs, psd, 'k-', alpha=0.5)
plt.plot(freqs[alpha_band], psd[alpha_band], 'b-', linewidth=2, label='Alpha')
plt.plot(freqs[gamma_band], psd[gamma_band], 'r-', linewidth=2, label='Gamma')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD')
plt.xlim(0, 100)
plt.legend()


In [ ]:
# Comparar distribuciones de pesos
print("Python S excitatorio:", np.mean(S[:, :800]), np.std(S[:, :800]))
print("Python S inhibitorio:", np.mean(S[:, 800:]), np.std(S[:, 800:]))

In [ ]:
# Python original - contar spikes por timestep
print("Python spikes/ms:", np.mean(firings[:,0]), np.max(firings[:,0]))
